# Parallelized version

In [1]:
import math
import random
import numpy as np

## Initializing spark

In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
conf = SparkConf().setAppName("appName").setMaster("local")
sc = SparkContext()

## Functions

### Auxiliar functions

In [4]:
def trainTestSplit(dataset):
    train = dataset.filter(lambda x: x[2]==1)
    train = train.map(lambda x: (x[0], x[1]))
    test = dataset.filter(lambda x: x[2]==0)
    test = test.map(lambda x: (x[0], x[1]))
    return (train, test)

In [5]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [6]:
def rdd_cost_function(RDD_Xyyhat, lambda_ref, m, w):
    y = RDD_Xyyhat[1]
    y_hat = RDD_Xyyhat[2]
    temp = (-1/m) * (
        y * np.log(y_hat) + (1 - y)
        * np.log(1 - y_hat)
    )
    cost = temp + (lambda_ref/(2*m))*sum([i*i for i in w.value])
    return cost

In [7]:
def str_to_number_list(line):
    numbers = line.replace(" ","").split(",")
    numbers = [float(number) for number in numbers]
    
    # Separate X and y
    numbers = (numbers[:-1], numbers[-1])
    
    return numbers

In [8]:
def sum_list_values(list1, list2):
    return [n1 + n2 for n1, n2 in zip(list1, list2)]

In [9]:
def sum_minus_mean_squared(numbers):
    return [(number-means.value[x])**2 for x,number in enumerate(numbers)]

In [10]:
def rdd_mean_by_column(RDD_Xy, m):
    means = RDD_Xy.map(lambda x: x[0]).reduce(sum_list_values)
    means = [x/m for x in means]
    return means

In [11]:
def rdd_mean_by_column(RDD_Xy, m):
    means = RDD_Xy.map(lambda x: x[0]).reduce(sum_list_values)
    means = [x/m for x in means]
    return means

In [12]:
def rdd_std_by_column(RDD_Xy, m):
    stds = RDD_Xy.map(lambda x: x[0]).map(sum_minus_mean_squared).reduce(sum_list_values)
    stds = [math.sqrt(x/m) for x in stds]
    return stds

In [13]:
def multiply_RDDXy_by_w(Xy):
    global b
    tot = 0
    for xi, wi in zip(Xy[0], w.value):
        tot += xi * wi
    tot += b
    return (Xy[0], Xy[1], sigmoid(tot))

In [14]:
def calculate_dw(RDD_Xyyhat):
    dw = []
    for x in RDD_Xyyhat[0]:
        dw.append((RDD_Xyyhat[2]-RDD_Xyyhat[1])*x)
    return dw

### Mandatory functions

In [15]:
def readFile(filename):
    dataset = sc.textFile(filename)
    dataset = dataset.map(str_to_number_list)
    return dataset.sample(False, 1)

In [16]:
def normalize(RDD_Xy):
    m = RDD_Xy.count()
    global means
    means = sc.broadcast(rdd_mean_by_column(RDD_Xy, m))
    stds = sc.broadcast(rdd_std_by_column(RDD_Xy, m))
    norm_rdd = RDD_Xy.map(lambda values: ([(x - mean) / std for x, mean, std in zip(values[0], means.value, stds.value)], values[1]))
    return norm_rdd

In [27]:
def train(RDD_Xy, iterations, learning_rate, lambda_reg):
    global b
    global m
    m = RDD_Xy.count()
    n = len(RDD_Xy.take(1)[0][0])
    global w
    w_temp = w.value.copy()
    for it in range(iterations):
        RDD_Xyyhat = RDD_Xy.map(multiply_RDDXy_by_w)
        print(f"Cost for it {it}:", RDD_Xyyhat.map(lambda x: rdd_cost_function(x, lambda_reg, m, w)).reduce(lambda x,y: x+y))
        dw=[0 for i in range(0,n)]
        for cl in range(n):
                X_cl = RDD_Xyyhat.map(lambda x: (x[2]-x[1])*x[0][cl]).reduce(lambda x, y: x+y)
                dw[cl] = (1/m)*(X_cl)+(lambda_reg/m)*w_temp[cl]
                w_temp[cl] -= learning_rate * dw[cl]
        w = sc.broadcast(w_temp)
        db = (1/m)*RDD_Xyyhat.map(lambda x: x[2]-x[1]).reduce(lambda x, y: x+y)
        b -= learning_rate * db
    return w, b

In [18]:
def accuracy(ws, b, RDD_Xy):
    total = RDD_Xy.count()
    y_and_y_hat = RDD_Xy.map(lambda x: (x[1], predict(ws, x[0], b)))
    correct = y_and_y_hat.map(lambda x: 1 if x[0]==x[1] else 0).reduce(lambda x, y: x+y)
    print(f"Accuacy {100*correct/total}%")
    return correct/total

In [19]:
def predict(w, X, b):
    tot = 0
    for xi, wi in zip(X, w.value):
        tot += xi * wi
    tot += b
    val = sigmoid(tot)
    if (val>=0.5):
        return 1.0
    return 0.0

## Testing

In [20]:
RDD_Xy = readFile("../data/botnet_tot_syn_l.csv")
RDD_Xy.count()

1000000

In [21]:
RDD_Xy_normalized = normalize(RDD_Xy)

In [28]:
RDD_Xy_presplit = RDD_Xy_normalized.map(lambda x: (x[0], x[1], 1 if random.random() < 0.75 else 0))
train_data, test_data = trainTestSplit(RDD_Xy_presplit)

#### Initialize some stuff before training

In [23]:
np.random.seed(33)

global b  
b = 0

global n
n = len(RDD_Xy.take(1)[0][0])

global w
w = sc.broadcast(np.random.rand(n))

#### Training

In [24]:
w_final, b = train(train_data, 10, 1.5, 0)

Cost for it 0: 1.4442800855775249
Cost for it 1: 0.7363844603637598
Cost for it 2: 0.4504741517348974
Cost for it 3: 0.3453871852072486
Cost for it 4: 0.2969814789754337
Cost for it 5: 0.2699574299731541
Cost for it 6: 0.2521263244964357
Cost for it 7: 0.23891588347471807
Cost for it 8: 0.22913243365273137
Cost for it 9: 0.22167804788406148
[-0.22287614 -0.79930396 -0.27938856 -0.19861265 -1.13137055  0.82767267
  0.12678874 -1.00074084  0.45341483  0.85839129  0.65360624]


#### Accuracy

In [25]:
acc = accuracy(w_final, b, test_data)

Accuacy 93.22270490126472%
